# Прогнозирование оттока клиентов банка

## Загрузка данных

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.utils import shuffle
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler 
from sklearn.metrics import roc_auc_score

from sklearn.metrics import f1_score
#!pip install pandas_profiling
#import pandas_profiling

In [2]:
try:df=pd.read_csv('/datasets/Churn.csv')
except: df=pd.read_csv('/Users/knovikova/Downloads/Churn.csv')
    
display(df.info())
display()
display(df.head(10))
display()
display(df.describe())
#pandas_profiling.ProfileReport(df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


None

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0
5,6,15574012,Chu,645,Spain,Male,44,8.0,113755.78,2,1,0,149756.71,1
6,7,15592531,Bartlett,822,France,Male,50,7.0,0.00,2,1,1,10062.80,0
7,8,15656148,Obinna,376,Germany,Female,29,4.0,115046.74,4,1,0,119346.88,1
8,9,15792365,He,501,France,Male,44,4.0,142051.07,2,0,1,74940.50,0
9,10,15592389,H?,684,France,Male,27,2.0,134603.88,1,1,1,71725.73,0


,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.00000,1.000000e+04,10000.000000,10000.000000,9091.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,5000.50000,1.569094e+07,650.528800,38.921800,4.997690,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,2886.89568,7.193619e+04,96.653299,10.487806,2.894723,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,1.00000,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,2500.75000,1.562853e+07,584.000000,32.000000,2.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,5000.50000,1.569074e+07,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,7500.25000,1.575323e+07,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,10000.00000,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


**Легенда**

**Признаки**

- RowNumber — индекс строки в данных
- CustomerId — уникальный идентификатор клиента
- Surname — фамилия
- CreditScore — кредитный рейтинг
- Geography — страна проживания
- Gender — пол
- Age — возраст
- Tenure — сколько лет человек является клиентом банка
- Balance — баланс на счёте
- NumOfProducts — количество продуктов банка, используемых клиентом
- HasCrCard — наличие кредитной карты
- IsActiveMember — активность клиента
- EstimatedSalary — предполагаемая зарплата

**Целевой признак**

- Exited — факт ухода клиента

## Изучение и предобработка данных

### Изменение регистра столбцов

In [3]:
df.columns = df.columns.str.lower()
print(df.columns)

Index(['rownumber', 'customerid', 'surname', 'creditscore', 'geography',
       'gender', 'age', 'tenure', 'balance', 'numofproducts', 'hascrcard',
       'isactivemember', 'estimatedsalary', 'exited'],
      dtype='object')


### Заполнение пропусков

Пропуски имеются в столбце Tenure, возможно, потому что данные клиенты обслуживаются в банке менее года. Заменим на 0 пропущенные значения.

In [4]:
print(df['tenure'].value_counts()) 

1.0     952
2.0     950
8.0     933
3.0     928
5.0     927
7.0     925
4.0     885
9.0     882
6.0     881
10.0    446
0.0     382
Name: tenure, dtype: int64


In [5]:
#заменим пропуски средним
df['tenure'] = df['tenure'].fillna(df['tenure'].mean())

In [6]:
print(df.duplicated().sum()) 

0


### Кодирование категориальных данных

Для начала уберем из анализа столбцы с фамилиями и номером строки - это ненужные данные

In [7]:
df = df.drop(['rownumber','surname','customerid'],axis=1)

Используем метод прямого кодирования для категориальных данных

In [8]:
df = pd.get_dummies(df,drop_first=True)
display(df.head())

,creditscore,age,tenure,balance,numofproducts,hascrcard,isactivemember,estimatedsalary,exited,geography_Germany,geography_Spain,gender_Male
0,619,42,2.0,0.00,1,1,1,101348.88,1,0,0,0
1,608,41,1.0,83807.86,1,0,1,112542.58,0,0,1,0
2,502,42,8.0,159660.80,3,1,0,113931.57,1,0,0,0
3,699,39,1.0,0.00,2,0,0,93826.63,0,0,0,0
4,850,43,2.0,125510.82,1,1,1,79084.10,0,0,1,0


# 2. Построение модели

Построим несколько моделей и выберем ту, которая даст наилучший результат.

In [9]:
print('Количество ушедших клиентов:',df[df['exited']==1]['exited'].count())
print('Количество оставшихся клиентов:',df[df['exited']==0]['exited'].count())
print('Дисбаланс данных в выборке составляет',df[df['exited']==0]['exited'].count()/df[df['exited']==1]['exited'].count())

#разделим выборку
features_train, features_test, target_train, target_test = train_test_split(
    df.drop(['exited'], axis=1),
    df['exited'],
    test_size=0.2,
    random_state=12345
)
features_train, features_valid, target_train, target_valid = train_test_split(
    features_train,
    target_train,
    test_size=0.25,
    random_state=12345
)

#проведем масштабирование методом стандартизации данных
scaler = StandardScaler()
columns = ['creditscore','age', 'tenure', 'balance', 'numofproducts', 'hascrcard','isactivemember', 'estimatedsalary']
scaler.fit(features_train[columns]) 
features_train[columns] = scaler.transform(features_train[columns])
features_valid[columns] = scaler.transform(features_valid[columns])
features_test[columns] = scaler.transform(features_test[columns])
pd.options.mode.chained_assignment = None

#функция для увеличения выборки ушедших клиентов
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_big = pd.concat([features_zeros] + [features_ones] * repeat)
    target_big = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_big, target_big = shuffle(
        features_big, target_big, random_state=12345)
    
    return features_big, target_big

features_big, target_big = upsample(features_train, target_train, 4)

Количество ушедших клиентов: 2037
Количество оставшихся клиентов: 7963
Дисбаланс данных в выборке составляет 3.9091801669121256


Построим три вида моделей, где отдельно применим к каждой три вида изменения дисбаланса.

**Логистическая модель:**

In [10]:
#построим логистическую модель без учета балансировки
model_log_1 = LogisticRegression(random_state=12345, solver='newton-cg')
model_log_1.fit(features_train, target_train)
predicted_log_1 = model_log_1.predict(features_valid)

probabilities_valid_1 = model_log_1.predict_proba(features_valid)
probabilities_one_valid_1 = probabilities_valid_1[:, 1]

f1_log_1 = f1_score(target_valid, predicted_log_1)
auc_roc_log_1 = roc_auc_score(target_valid,probabilities_one_valid_1)

print("F1 несбалансированное:", f1_log_1)
print("ROC-AUC несбалансированное:", auc_roc_log_1)

#построим логистическую модель с учетом балансировки
model_log_2 = LogisticRegression(random_state=12345, solver='newton-cg',class_weight='balanced')
model_log_2.fit(features_big, target_big)
predicted_log_2 = model_log_2.predict(features_valid)

probabilities_valid_2 = model_log_2.predict_proba(features_valid)
probabilities_one_valid_2 = probabilities_valid_2[:, 1]

f1_log_2 = f1_score(target_valid, predicted_log_2)
auc_roc_log_2 = roc_auc_score(target_valid,probabilities_one_valid_2)

print("F1 сбалансированное:", f1_log_2)
print("ROC-AUC сбалансированное:", auc_roc_log_2)

#Найдем значения вероятностей классов для валидационной выборки
probabilities_log_3 = model_log_1.predict_proba(features_valid)
#Значения вероятностей класса «1» 
probabilities_one_log_3 = probabilities_log_3[:, 1]

f1_3_log_list = []
auc_roc_log_3_list = []
for threshold in np.arange(0.1, 0.5, 0.02):
    
    predicted_log_3 = probabilities_one_log_3 > threshold 
    f1_log_3 = f1_score(target_valid,predicted_log_3)
    auc_roc_3 = roc_auc_score(target_valid,predicted_log_3)
    f1_3_log_list.append(f1_log_3)
    auc_roc_log_3_list.append(auc_roc_3)
    
    print("Порог = {:.2f} | F1 = {:.3f}, AUC-ROC = {:.2f}".format(
        threshold, f1_log_3,auc_roc_3))

#построим логистическую модель обученную на расширенной выборке
model_log_4 = LogisticRegression(random_state=12345, solver='newton-cg')
model_log_4.fit(features_big, target_big)
predicted_log_4 = model_log_4.predict(features_valid)
    
probabilities_valid_4 = model_log_4.predict_proba(features_valid)
probabilities_one_valid_4 = probabilities_valid_4[:, 1]

f1_log_4 = f1_score(target_valid, predicted_log_4)
auc_roc_log_4 = roc_auc_score(target_valid,probabilities_one_valid_4)

print("F1 несбалансированное на расширенной выборке:", f1_log_4) 
print("ROC-AUC несбалансированное на расширенной выборке:", auc_roc_log_4)

#соберем список f1 и AUC-ROC
f1_log_list = [f1_log_1,f1_log_2,max(f1_3_log_list),f1_log_4]
auc_roc_log_list = [auc_roc_log_1,auc_roc_log_2,max(auc_roc_log_3_list),auc_roc_log_4]

print()
print('У логистической модели наибольшее значение F1 показала модель с подкруткой порога:',max(f1_log_list))

F1 несбалансированное: 0.30131826741996237
ROC-AUC несбалансированное: 0.7702739863205529
F1 сбалансированное: 0.47457627118644063
ROC-AUC сбалансированное: 0.7725819757470368
Порог = 0.10 | F1 = 0.413, AUC-ROC = 0.65
Порог = 0.12 | F1 = 0.429, AUC-ROC = 0.67
Порог = 0.14 | F1 = 0.446, AUC-ROC = 0.69
Порог = 0.16 | F1 = 0.462, AUC-ROC = 0.70
Порог = 0.18 | F1 = 0.460, AUC-ROC = 0.69
Порог = 0.20 | F1 = 0.466, AUC-ROC = 0.69
Порог = 0.22 | F1 = 0.474, AUC-ROC = 0.69
Порог = 0.24 | F1 = 0.477, AUC-ROC = 0.69
Порог = 0.26 | F1 = 0.475, AUC-ROC = 0.68
Порог = 0.28 | F1 = 0.479, AUC-ROC = 0.68
Порог = 0.30 | F1 = 0.490, AUC-ROC = 0.69
Порог = 0.32 | F1 = 0.494, AUC-ROC = 0.68
Порог = 0.34 | F1 = 0.483, AUC-ROC = 0.68
Порог = 0.36 | F1 = 0.471, AUC-ROC = 0.67
Порог = 0.38 | F1 = 0.455, AUC-ROC = 0.66
Порог = 0.40 | F1 = 0.436, AUC-ROC = 0.65
Порог = 0.42 | F1 = 0.416, AUC-ROC = 0.63
Порог = 0.44 | F1 = 0.379, AUC-ROC = 0.62
Порог = 0.46 | F1 = 0.359, AUC-ROC = 0.61
Порог = 0.48 | F1 = 0.329,

**Модель дерева решений:**

In [11]:
#модель без изменения дисбаланса данных
f1_dtc_1 = 0
model_dtc_1 = None
depth_dtc_1 = 0
for est in tqdm(range(1, 11)):
    for depth in range(1,16):
        model = DecisionTreeClassifier(random_state=12345,criterion='entropy', max_depth=depth)
        model.fit(features_train, target_train)
        predicted_valid = model.predict(features_valid)
        f1 = f1_score(target_valid, predicted_valid)
        if f1 > f1_dtc_1:
            f1_dtc_1 = f1
            model_dtc_1 = model
            depth_dtc_1 = depth
probabilities_dtc_1 = model_dtc_1.predict_proba(features_valid)
probabilities_one_dtc_1 = probabilities_dtc_1[:, 1]
auc_roc_dtc_1 = roc_auc_score(target_valid,probabilities_one_dtc_1)
print(f' Наилучшее значение f1 небалансированного дерева получается при глубине {depth_dtc_1}, f1 = {f1_dtc_1}, AUC-ROC = {auc_roc_dtc_1}')

#модель дерева с учетом балансировки
f1_dtc_2 = 0
model_dtc_2 = None
depth_dtc_2 = 0
for est in tqdm(range(1, 11)):
    for depth in range(1,16):
        model = DecisionTreeClassifier(random_state=12345,criterion='entropy', max_depth=depth, class_weight='balanced')
        model.fit(features_train, target_train)
        predicted_valid = model.predict(features_valid)
        f1 = f1_score(target_valid, predicted_valid)
        if f1 > f1_dtc_2:
            f1_dtc_2 = f1
            model_dtc_2 = model
            depth_dtc_2 = depth
probabilities_dtc_2 = model_dtc_2.predict_proba(features_valid)
probabilities_one_dtc_2 = probabilities_dtc_2[:, 1]
auc_roc_dtc_2 = roc_auc_score(target_valid,probabilities_one_dtc_2)
print(f' Наилучшее значение f1 сбалансированного дает дерево при глубине {depth_dtc_2}, f1 = {f1_dtc_2}, AUC-ROC = {auc_roc_dtc_2}')

#Найдем значения вероятностей классов для валидационной выборки
probabilities_dtc_3 = model_dtc_1.predict_proba(features_valid)
#Значения вероятностей класса «1» 
probabilities_one_dtc_3 = probabilities_dtc_3[:, 1]

f1_3_dtc_list=[]
auc_roc_dtc_3_list = []
for threshold in np.arange(0.4, 0.6, 0.02):
    predicted_dtc_3 = probabilities_one_dtc_3 > threshold 
    f1_dtc_3 = f1_score(target_valid,predicted_dtc_3)
    auc_roc_dtc_3 = roc_auc_score(target_valid,predicted_dtc_3)
    f1_3_dtc_list.append(f1_dtc_3)
    auc_roc_dtc_3_list.append(auc_roc_dtc_3)
    print("Порог = {:.2f} | F1 = {:.3f}, AUC-ROC = {:.2f}".format(
        threshold, f1_dtc_3,auc_roc_dtc_3))
    
#модель дерева обученная на расширенной выборке
f1_dtc_4 = 0
model_dtc_4 = None
depth_dtc_4 = 0
for est in tqdm(range(1, 11)):
    for depth in range(1,16):
        model = DecisionTreeClassifier(random_state=12345,criterion='entropy', max_depth=depth)
        model.fit(features_big, target_big)
        predicted_valid = model.predict(features_valid)
        f1 = f1_score(target_valid, predicted_valid)
        if f1 > f1_dtc_4:
            f1_dtc_4 = f1
            model_dtc_4 = model
            depth_dtc_4 = depth
probabilities_dtc_4 = model_dtc_4.predict_proba(features_valid)
probabilities_one_dtc_4 = probabilities_dtc_4[:, 1]
auc_roc_dtc_4 = roc_auc_score(target_valid,probabilities_one_dtc_4)
print(f' Наилучшее значение f1 на расширенной выборке дает дерево при глубине {depth_dtc_4}, f1 = {f1_dtc_4},AUC-ROC = {auc_roc_dtc_4}')

#соберем список f1 и AUC-ROC
f1_dtc_list = [f1_dtc_1,f1_dtc_2,max(f1_3_dtc_list),f1_dtc_4]
auc_roc_dtc_list = [auc_roc_dtc_1,auc_roc_dtc_2,max(auc_roc_dtc_3_list),auc_roc_dtc_4]
print()
print('У модели дерева решений наибольшее значение F1 показала модель с подкруткой порога:',max(f1_dtc_list))


100%|██████████| 10/10 [00:03<00:00,  3.12it/s]


 Наилучшее значение f1 небалансированного дерева получается при глубине 7, f1 = 0.5342019543973942, AUC-ROC = 0.8206809840427647


100%|██████████| 10/10 [00:03<00:00,  2.88it/s]


 Наилучшее значение f1 сбалансированного дает дерево при глубине 8, f1 = 0.5540106951871657, AUC-ROC = 0.8030126255922965
Порог = 0.40 | F1 = 0.570, AUC-ROC = 0.73
Порог = 0.42 | F1 = 0.570, AUC-ROC = 0.73
Порог = 0.44 | F1 = 0.550, AUC-ROC = 0.70
Порог = 0.46 | F1 = 0.550, AUC-ROC = 0.70
Порог = 0.48 | F1 = 0.550, AUC-ROC = 0.70
Порог = 0.50 | F1 = 0.534, AUC-ROC = 0.69
Порог = 0.52 | F1 = 0.534, AUC-ROC = 0.69
Порог = 0.54 | F1 = 0.534, AUC-ROC = 0.69
Порог = 0.56 | F1 = 0.492, AUC-ROC = 0.67
Порог = 0.58 | F1 = 0.492, AUC-ROC = 0.67


100%|██████████| 10/10 [00:03<00:00,  2.60it/s]

 Наилучшее значение f1 на расширенной выборке дает дерево при глубине 7, f1 = 0.5545893719806764,AUC-ROC = 0.8198043613370444

У модели дерева решений наибольшее значение F1 показала модель с подкруткой порога: 0.5695364238410596


**Модель случайного леса:**

In [12]:
#несбалансированная модель случайного леса
model_rfc_1 = None
f1_rfc_1 = 0
depth_rfc_1 = 0
est_rfc_1 = 0
for est in range(40, 71, 10):
    for depth in range (8, 16):
        model = RandomForestClassifier(random_state=12345, n_estimators=est,max_depth=depth)
        model.fit(features_train, target_train) 
        predicted_rfc = model.predict(features_valid)
        result = f1_score(target_valid, predicted_rfc)
        if result > f1_rfc_1:
            model_rfc_1 = model
            f1_rfc_1 = result
            depth_rfc_1 = est
            est_rfc_1 = depth
probabilities_rfc_1 = model_rfc_1.predict_proba(features_valid)
probabilities_one_rfc_1 = probabilities_rfc_1[:, 1]
auc_roc_rfc_1 = roc_auc_score(target_valid,probabilities_one_rfc_1)
print(f'Несбалансированная модель случайного леса имеет глубину {est_rfc_1} и количество деревьев {depth_rfc_1}, значение F1 равно {f1_rfc_1},AUC-ROC = {auc_roc_dtc_1}')

#Сбалансированная модель случайного леса
model_rfc_2 = None
f1_rfc_2 = 0
depth_rfc_2 = 0
est_rfc_2 = 0
for est in range(40, 71, 10):
    for depth in range (8, 16):
        model = RandomForestClassifier(random_state=12345, n_estimators=est,max_depth=depth,class_weight='balanced')
        model.fit(features_train, target_train) 
        predicted_rfc = model.predict(features_valid)
        result = f1_score(target_valid, predicted_rfc)
        if result > f1_rfc_2:
            model_rfc_2 = model
            f1_rfc_2 = result
            depth_rfc_2 = est
            est_rfc_2 = depth
probabilities_rfc_2 = model_rfc_2.predict_proba(features_valid)
probabilities_one_rfc_2 = probabilities_rfc_2[:, 1]
auc_roc_rfc_2 = roc_auc_score(target_valid,probabilities_one_rfc_2)
print(f'Cбалансированная модель случайного леса имеет глубину {est_rfc_2} и количество деревьев {depth_rfc_2}, значение F1 равно {f1_rfc_2},AUC-ROC = {auc_roc_dtc_2}')

#Найдем значения вероятностей классов для валидационной выборки
probabilities_rfc_3 = model_rfc_1.predict_proba(features_valid)
#Значения вероятностей класса «1» 
probabilities_one_rfc_3 = probabilities_rfc_3[:, 1]
f1_3_rfc_list=[]
auc_roc_rfc_3_list = []
for threshold in np.arange(0.3, 0.44, 0.02):
    predicted_rfc_3 = probabilities_one_rfc_3 > threshold 
    f1_rfc_3 = f1_score(target_valid,predicted_rfc_3)
    auc_roc_rfc_3 = roc_auc_score(target_valid,predicted_rfc_3)
    f1_3_rfc_list.append(f1_rfc_3)
    auc_roc_rfc_3_list.append(auc_roc_rfc_3)
    print("Порог = {:.2f} | F1 = {:.3f}, AUC-ROC = {:.2f}".format(
        threshold, f1_rfc_3,auc_roc_rfc_3))
best_threshold = 0.4

#модель случайного леса обученная на расширенной выборке
model_rfc_4 = None
f1_rfc_4 = 0
depth_rfc_4 = 0
est_rfc_4 = 0
for est in range(70, 91, 10):
    for depth in range (8, 16):
        model = RandomForestClassifier(random_state=12345, n_estimators=est,max_depth=depth)
        model.fit(features_big, target_big) 
        predicted_rfc = model.predict(features_valid)
        result = f1_score(target_valid, predicted_rfc)
        if result > f1_rfc_4:
            model_rfc_4 = model
            f1_rfc_4 = result
            depth_rfc_4 = est
            est_rfc_4 = depth
probabilities_rfc_4 = model_rfc_4.predict_proba(features_valid)
probabilities_one_rfc_4 = probabilities_rfc_4[:, 1]
auc_roc_rfc_4 = roc_auc_score(target_valid,probabilities_one_rfc_4)
print(f'Несбалансированная модель случайного леса на расширенной выборке имеет глубину {est_rfc_4} и количество деревьев {depth_rfc_4}, значение F1 равно {f1_rfc_4},AUC-ROC = {auc_roc_dtc_4}')


#соберем список f1 и AUC-ROC
f1_rfc_list = [f1_rfc_1,f1_rfc_2,max(f1_3_rfc_list),f1_rfc_4]
auc_roc_rfc_list = [auc_roc_rfc_1,auc_roc_rfc_2,max(auc_roc_rfc_3_list),auc_roc_rfc_4]
print()
print('У модели случайного леса наибольшее значение F1 показала модель с подкруткой порога:',max(f1_rfc_list))

Несбалансированная модель случайного леса имеет глубину 14 и количество деревьев 40, значение F1 равно 0.5678233438485804,AUC-ROC = 0.8206809840427647
Cбалансированная модель случайного леса имеет глубину 8 и количество деревьев 40, значение F1 равно 0.5914844649021864,AUC-ROC = 0.8030126255922965
Порог = 0.30 | F1 = 0.586, AUC-ROC = 0.75
Порог = 0.32 | F1 = 0.597, AUC-ROC = 0.75
Порог = 0.34 | F1 = 0.589, AUC-ROC = 0.75
Порог = 0.36 | F1 = 0.589, AUC-ROC = 0.74
Порог = 0.38 | F1 = 0.600, AUC-ROC = 0.74
Порог = 0.40 | F1 = 0.606, AUC-ROC = 0.74
Порог = 0.42 | F1 = 0.594, AUC-ROC = 0.73
Порог = 0.44 | F1 = 0.590, AUC-ROC = 0.73
Несбалансированная модель случайного леса на расширенной выборке имеет глубину 10 и количество деревьев 70, значение F1 равно 0.5984251968503937,AUC-ROC = 0.8198043613370444

У модели случайного леса наибольшее значение F1 показала модель с подкруткой порога: 0.6061452513966481


# 3 Тестирование решения. Вывод

Соберем данные в таблицу

In [13]:
table = {'Модель': ['Логистическая модель 1','Логистическая модель 2','Логистическая модель 3', 'Логистическая модель 4',
          'Модель дерева решений 1','Модель дерева решений 2','Модель дерева решений 3','Модель дерева решений 4',
         'Модель случайного леса 1','Модель случайного леса 2', 'Модель случайного леса 3', 'Модель случайного леса 4'],
        'F1':[f1_log_1,f1_log_2,max(f1_3_log_list),f1_log_4,f1_dtc_1,f1_dtc_2,max(f1_3_dtc_list),f1_dtc_4,f1_rfc_1,f1_rfc_2,max(f1_3_rfc_list),f1_rfc_4],
        'AUC-ROC':[auc_roc_log_1,auc_roc_log_2,max(auc_roc_log_3_list),auc_roc_log_4,auc_roc_dtc_1,auc_roc_dtc_2,max(auc_roc_dtc_3_list),auc_roc_dtc_4,auc_roc_rfc_1,auc_roc_rfc_2,max(auc_roc_rfc_3_list),auc_roc_rfc_4]}

table = pd.DataFrame(table)

display(table)

,Модель,F1,AUC-ROC
0,Логистическая модель 1,0.301318,0.770274
1,Логистическая модель 2,0.474576,0.772582
2,Логистическая модель 3,0.494179,0.697227
3,Логистическая модель 4,0.476693,0.772564
4,Модель дерева решений 1,0.534202,0.820681
5,Модель дерева решений 2,0.554011,0.803013
6,Модель дерева решений 3,0.569536,0.728634
7,Модель дерева решений 4,0.554589,0.819804
8,Модель случайного леса 1,0.567823,0.832996
9,Модель случайного леса 2,0.591484,0.852498


Наибольшее значение F1 показывает модель случайного леса. Второй, третий и четвертый варианты модели были очень близки по значению f1, поэтому сравним все три варианта.

In [14]:
#тест Модели дерева решений 2 - сбалансированные данные 
pred_test_2 = model_rfc_2.predict(features_test)
f1_test_2 = f1_score(target_test, pred_test_2)
auc_roc_test_2 = roc_auc_score(target_test, pred_test_2)

#тест Модели дерева решений 3 - подкручивание порога
pred_test_3 = model_rfc_1.predict_proba(features_test)[:, 1]
predictions_3 = pred_test_3 > best_threshold
f1_test_3 = f1_score(target_test, predictions_3)
auc_roc_test_3 = roc_auc_score(target_test, pred_test_3)

#тест Модели дерева решений 4 - обучение на расширенной выборке
pred_test_4 = model_rfc_4.predict(features_test)
f1_test_4 = f1_score(target_test, pred_test_4)
auc_roc_test_4 = roc_auc_score(target_test, pred_test_3)

print(f'Модель случайного леса 2 на тестовых данных имеет значение F1:{f1_test_2:.2}, AUC-ROC:{auc_roc_test_2:.2}')
print(f'Модель случайного леса 4 на тестовых данных имеет значение F1:{f1_test_3:.2}, AUC-ROC:{auc_roc_test_3:.2} ')
print(f'Модель случайного леса 4 на тестовых данных имеет значение F1:{f1_test_4:.2}, AUC-ROC:{auc_roc_test_4:.2} ')

Модель случайного леса 2 на тестовых данных имеет значение F1:0.65, AUC-ROC:0.79
Модель случайного леса 4 на тестовых данных имеет значение F1:0.62, AUC-ROC:0.86 
Модель случайного леса 4 на тестовых данных имеет значение F1:0.64, AUC-ROC:0.86 


Общий вывод:
- Построено три вида моделей четырься различными подходами: без учета дисбаланса классов, с балансировкой, с расширением выборки и с подкруткой порога.
- Из 12 моделей были выбраны 2 модели случайного леса: с балансировкой и с расширением выборки
- Наилучший результат показала модель случайного леса 2, со сбалансированными данными выборки.